In [1]:
from ncsw_data.storage.cacs.sqlite_db import CaCSSQLiteDatabase

/home/haris/.conda/envs/ncsw-data-use-case-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
case_study_cacs_db_file_path = (
    "sqlite:////path/to/the/case_study_cacs_db.sqlite"
)

case_study_cacs_db = CaCSSQLiteDatabase(
    db_url=case_study_cacs_db_file_path
)

case_study_cacs_db.create_tables()

# Chemical Compounds

In [3]:
select_statement_result = case_study_cacs_db.execute_select_statement(
    select_statement_text="""
        SELECT
            COUNT(DISTINCT ac.id) AS number_of_archive_compounds,
            COUNT(DISTINCT wca.archive_compound_id) AS number_of_utilized_archive_compounds,
            100.0 * COUNT(DISTINCT wca.archive_compound_id) / COUNT(DISTINCT ac.id) AS percentage_of_utilized_archive_compounds,
            COUNT(DISTINCT wc.id) AS number_of_workbench_compounds
        FROM archive_compound AS ac
            LEFT JOIN workbench_compound_archive AS wca ON ac.id = wca.archive_compound_id
            LEFT JOIN workbench_compound AS wc ON wca.workbench_compound_id = wc.id;
    """
).fetchone()

print((
    "Number of Archive Chemical Compounds: {number_of_archive_compounds:,d}\n"
    "Number of Utilized Archive Chemical Compounds: {number_of_utilized_archive_compounds:,d}\n"
    "Percentage of Utilized Archive Chemical Compounds: {percentage_of_utilized_archive_compounds:f}\n"
    "Number of Workbench Chemical Compounds: {number_of_workbench_compounds:,d}"
).format(
    number_of_archive_compounds=select_statement_result[0],
    number_of_utilized_archive_compounds=select_statement_result[1],
    percentage_of_utilized_archive_compounds=select_statement_result[2],
    number_of_workbench_compounds=select_statement_result[3]
))

Number of Archive Chemical Compounds: 1,953,848
Number of Utilized Archive Chemical Compounds: 1,953,841
Percentage of Utilized Archive Chemical Compounds: 99.999642
Number of Workbench Chemical Compounds: 1,953,841


In [4]:
for select_statement_parameters_index, select_statement_parameters in enumerate([
    ("'v_building_block_bb_50'", "('')", ),
    ("'v_building_block_bb_40'", "('v_building_block_bb_50')", ),
    ("'v_building_block_bb_30'", "('v_building_block_bb_50', 'v_building_block_bb_40')", ),
]):
    select_statement_result = case_study_cacs_db.execute_select_statement(
        select_statement_text="""
            SELECT
                COUNT(DISTINCT ac_1.id) AS number_of_archive_compounds,
                COUNT(DISTINCT wca_1.archive_compound_id) AS number_of_utilized_archive_compounds
            FROM archive_compound AS ac_1
                JOIN archive_compound_source AS acs_1 ON ac_1.id = acs_1.archive_compound_id
                JOIN archive_source AS arso_1 ON acs_1.archive_source_id = arso_1.id
                LEFT JOIN workbench_compound_archive AS wca_1 ON ac_1.id = wca_1.archive_compound_id
            WHERE arso_1.version = {included_data_source_version:s} AND ac_1.id NOT IN (
                SELECT
                    acs_2.archive_compound_id
                FROM archive_compound AS ac_2
                    JOIN archive_compound_source AS acs_2 ON ac_2.id = acs_2.archive_compound_id
                    JOIN archive_source AS arso_2 ON acs_2.archive_source_id = arso_2.id
                WHERE arso_2.version IN {excluded_data_source_versions:s}
            );
        """.format(
            included_data_source_version=select_statement_parameters[0],
            excluded_data_source_versions=select_statement_parameters[1]
        )
    ).fetchone()

    print((
        "Data Source: zinc_{data_source_version:s}\n"
        "Number of Archive Chemical Compounds: {number_of_archive_compounds:,d}\n"
        "Number of Utilized Archive Chemical Compounds: {number_of_utilized_archive_compounds:,d}\n"
        "Percentage of Utilized Archive Chemical Compounds: {percentage_of_utilized_archive_compounds:f}"
    ).format(
        data_source_version=select_statement_parameters[0][1:-1],
        number_of_archive_compounds=select_statement_result[0],
        number_of_utilized_archive_compounds=select_statement_result[1],
        percentage_of_utilized_archive_compounds=100 * select_statement_result[1] / select_statement_result[0]
    ))

    if select_statement_parameters_index < 2:
        print("")

Data Source: zinc_v_building_block_bb_50
Number of Archive Chemical Compounds: 1,334,114
Number of Utilized Archive Chemical Compounds: 1,334,107
Percentage of Utilized Archive Chemical Compounds: 99.999475

Data Source: zinc_v_building_block_bb_40
Number of Archive Chemical Compounds: 616,442
Number of Utilized Archive Chemical Compounds: 616,442
Percentage of Utilized Archive Chemical Compounds: 100.000000

Data Source: zinc_v_building_block_bb_30
Number of Archive Chemical Compounds: 3,292
Number of Utilized Archive Chemical Compounds: 3,292
Percentage of Utilized Archive Chemical Compounds: 100.000000


In [5]:
for select_statement_parameters_index, select_statement_parameters in enumerate([
    ("'v_building_block_bb_50'", "('')", ),
    ("'v_building_block_bb_40'", "('v_building_block_bb_50')", ),
    ("'v_building_block_bb_30'", "('v_building_block_bb_50', 'v_building_block_bb_40')", ),
]):
    select_statement_result = case_study_cacs_db.execute_select_statement(
        select_statement_text="""
            SELECT
                COUNT(DISTINCT wc_1.id) AS number_of_workbench_compounds
            FROM workbench_compound AS wc_1
                JOIN workbench_compound_archive AS wca_1 ON wc_1.id = wca_1.workbench_compound_id
                JOIN archive_compound AS ac_1 ON wca_1.archive_compound_id = ac_1.id
                JOIN archive_compound_source AS acs_1 ON ac_1.id = acs_1.archive_compound_id
                JOIN archive_source AS arso_1 ON acs_1.archive_source_id = arso_1.id
            WHERE arso_1.version = {included_data_source_version:s} AND wc_1.id NOT IN (
                SELECT
                    wc_2.id
                FROM workbench_compound AS wc_2
                    JOIN workbench_compound_archive AS wca_2 ON wc_2.id = wca_2.workbench_compound_id
                    JOIN archive_compound AS ac_2 ON wca_2.archive_compound_id = ac_2.id
                    JOIN archive_compound_source AS acs_2 ON ac_2.id = acs_2.archive_compound_id
                    JOIN archive_source AS arso_2 ON acs_2.archive_source_id = arso_2.id
                WHERE arso_2.version IN {excluded_data_source_versions:s}
            );
        """.format(
            included_data_source_version=select_statement_parameters[0],
            excluded_data_source_versions=select_statement_parameters[1]
        )
    ).fetchone()

    print((
        "Data Source: zinc_{data_source_version:s}\n"
        "Number of Workbench Chemical Compounds: {number_of_workbench_compounds:,d}"
    ).format(
        data_source_version=select_statement_parameters[0][1:-1],
        number_of_workbench_compounds=select_statement_result[0]
    ))

    if select_statement_parameters_index < 2:
        print("")

Data Source: zinc_v_building_block_bb_50
Number of Workbench Chemical Compounds: 1,334,107

Data Source: zinc_v_building_block_bb_40
Number of Workbench Chemical Compounds: 616,442

Data Source: zinc_v_building_block_bb_30
Number of Workbench Chemical Compounds: 3,292


# Chemical Reactions

In [6]:
select_statement_result = case_study_cacs_db.execute_select_statement(
    select_statement_text="""
        SELECT
            COUNT(DISTINCT ar.id) AS number_of_archive_reactions,
            COUNT(DISTINCT wra.archive_reaction_id) AS number_of_utilized_archive_reactions,
            100.0 * COUNT(DISTINCT wra.archive_reaction_id) / COUNT(DISTINCT ar.id) AS percentage_of_utilized_archive_reactions,
            COUNT(DISTINCT wr.id) AS number_of_workbench_reactions,
            1.0 * COUNT(DISTINCT wr.id) / COUNT(DISTINCT ar.id) AS reaction_expansion_factor
        FROM archive_reaction AS ar
            LEFT JOIN workbench_reaction_archive AS wra ON ar.id = wra.archive_reaction_id
            LEFT JOIN workbench_reaction AS wr ON wra.workbench_reaction_id = wr.id;
    """
).fetchone()

print((
    "Number of Archive Chemical Reactions: {number_of_archive_reactions:,d}\n"
    "Number of Utilized Archive Chemical Reactions: {number_of_utilized_archive_reactions:,d}\n"
    "Percentage of Utilized Archive Chemical Reactions: {percentage_of_utilized_archive_reactions:f}\n"
    "Number of Workbench Chemical Reactions: {number_of_workbench_reactions:,d}\n"
    "Chemical Reaction Expansion Factor: {reaction_expansion_factor:f}"
).format(
    number_of_archive_reactions=select_statement_result[0],
    number_of_utilized_archive_reactions=select_statement_result[1],
    percentage_of_utilized_archive_reactions=select_statement_result[2],
    number_of_workbench_reactions=select_statement_result[3],
    reaction_expansion_factor=select_statement_result[4]
))

Number of Archive Chemical Reactions: 3,294,548
Number of Utilized Archive Chemical Reactions: 3,287,740
Percentage of Utilized Archive Chemical Reactions: 99.793356
Number of Workbench Chemical Reactions: 2,448,637
Chemical Reaction Expansion Factor: 0.743239


In [7]:
number_of_utilized_archive_reactions_per_data_source = list()

for select_statement_parameters_index, select_statement_parameters in enumerate([
    ("'uspto'", "('')", ),
    ("'ord'", "('uspto')", ),
    ("'crd'", "('uspto', 'ord')", ),
    ("'miscellaneous'", "('uspto', 'ord', 'crd')", ),
]):
    select_statement_result = case_study_cacs_db.execute_select_statement(
        select_statement_text="""
            SELECT
                COUNT(DISTINCT ar_1.id) AS number_of_archive_reactions,
                COUNT(DISTINCT wra_1.archive_reaction_id) AS number_of_utilized_archive_reactions
            FROM archive_reaction AS ar_1
                JOIN archive_reaction_source AS ars_1 ON ar_1.id = ars_1.archive_reaction_id
                JOIN archive_source AS arso_1 ON ars_1.archive_source_id = arso_1.id
                LEFT JOIN workbench_reaction_archive AS wra_1 ON ar_1.id = wra_1.archive_reaction_id
            WHERE arso_1.name = {included_data_source_name:s} AND ar_1.id NOT IN (
                SELECT
                    ars_2.archive_reaction_id
                FROM archive_reaction AS ar_2
                    JOIN archive_reaction_source AS ars_2 ON ar_2.id = ars_2.archive_reaction_id
                    JOIN archive_source AS arso_2 ON ars_2.archive_source_id = arso_2.id
                WHERE arso_2.name IN {excluded_data_source_names:s}
            );
        """.format(
            included_data_source_name=select_statement_parameters[0],
            excluded_data_source_names=select_statement_parameters[1]
        )
    ).fetchone()

    print((
        "Data Source: {data_source:s}\n"
        "Number of Archive Chemical Reactions: {number_of_archive_reactions:,d}\n"
        "Number of Utilized Archive Chemical Reactions: {number_of_utilized_archive_reactions:,d}\n"
        "Percentage of Utilized Archive Chemical Reactions: {percentage_of_utilized_archive_reactions:f}"
    ).format(
        data_source=select_statement_parameters[0][1:-1],
        number_of_archive_reactions=select_statement_result[0],
        number_of_utilized_archive_reactions=select_statement_result[1],
        percentage_of_utilized_archive_reactions=100 * select_statement_result[1] / select_statement_result[0]
    ))

    if select_statement_parameters_index < 3:
        print("")

    number_of_utilized_archive_reactions_per_data_source.append(
        select_statement_result[1]
    )

Data Source: uspto
Number of Archive Chemical Reactions: 1,484,441
Number of Utilized Archive Chemical Reactions: 1,484,069
Percentage of Utilized Archive Chemical Reactions: 99.974940

Data Source: ord
Number of Archive Chemical Reactions: 507,136
Number of Utilized Archive Chemical Reactions: 501,464
Percentage of Utilized Archive Chemical Reactions: 98.881562

Data Source: crd
Number of Archive Chemical Reactions: 1,300,871
Number of Utilized Archive Chemical Reactions: 1,300,112
Percentage of Utilized Archive Chemical Reactions: 99.941654

Data Source: miscellaneous
Number of Archive Chemical Reactions: 2,100
Number of Utilized Archive Chemical Reactions: 2,095
Percentage of Utilized Archive Chemical Reactions: 99.761905


In [8]:
for select_statement_parameters_index, select_statement_parameters in enumerate([
    ("'uspto'", "('')", ),
    ("'ord'", "('uspto')", ),
    ("'crd'", "('uspto', 'ord')", ),
    ("'miscellaneous'", "('uspto', 'ord', 'crd')", ),
]):
    select_statement_result = case_study_cacs_db.execute_select_statement(
        select_statement_text="""
            SELECT
                COUNT(DISTINCT wr_1.id) AS number_of_workbench_reactions
            FROM workbench_reaction AS wr_1
                JOIN workbench_reaction_archive AS wra_1 ON wr_1.id = wra_1.workbench_reaction_id
                JOIN archive_reaction AS ar_1 ON wra_1.archive_reaction_id = ar_1.id
                JOIN archive_reaction_source AS ars_1 ON ar_1.id = ars_1.archive_reaction_id
                JOIN archive_source AS arso_1 ON ars_1.archive_source_id = arso_1.id
            WHERE arso_1.name = {included_data_source_name:s} AND wr_1.id NOT IN (
                SELECT
                    wr_2.id
                FROM workbench_reaction AS wr_2
                    JOIN workbench_reaction_archive AS wra_2 ON wr_2.id = wra_2.workbench_reaction_id
                    JOIN archive_reaction AS ar_2 ON wra_2.archive_reaction_id = ar_2.id
                    JOIN archive_reaction_source AS ars_2 ON ar_2.id = ars_2.archive_reaction_id
                    JOIN archive_source AS arso_2 ON ars_2.archive_source_id = arso_2.id
                WHERE arso_2.name IN {excluded_data_source_names:s}
            );
        """.format(
            included_data_source_name=select_statement_parameters[0],
            excluded_data_source_names=select_statement_parameters[1]
        )
    ).fetchone()

    print((
        "Data Source: {data_source:s}\n"
        "Number of Workbench Chemical Reactions: {number_of_workbench_reactions:,d}\n"
        "Chemical Reaction Expansion Factor: {reaction_expansion_factor:f}"
    ).format(
        data_source=select_statement_parameters[0][1:-1],
        number_of_workbench_reactions=select_statement_result[0],
        reaction_expansion_factor=select_statement_result[0] / number_of_utilized_archive_reactions_per_data_source[select_statement_parameters_index]
    ))

    if select_statement_parameters_index < 3:
        print("")

Data Source: uspto
Number of Workbench Chemical Reactions: 1,437,750
Chemical Reaction Expansion Factor: 0.968789

Data Source: ord
Number of Workbench Chemical Reactions: 260,924
Chemical Reaction Expansion Factor: 0.520324

Data Source: crd
Number of Workbench Chemical Reactions: 747,264
Chemical Reaction Expansion Factor: 0.574769

Data Source: miscellaneous
Number of Workbench Chemical Reactions: 2,699
Chemical Reaction Expansion Factor: 1.288305


# Chemical Reaction Patterns

In [9]:
select_statement_result = case_study_cacs_db.execute_select_statement(
    select_statement_text="""
        SELECT
            COUNT(DISTINCT arp.id) AS number_of_archive_reaction_patterns,
            COUNT(DISTINCT wrpa.archive_reaction_pattern_id) AS number_of_utilized_archive_reaction_patterns,
            100.0 * COUNT(DISTINCT wrpa.archive_reaction_pattern_id) / COUNT(DISTINCT arp.id) AS percentage_of_utilized_archive_reaction_patterns,
            COUNT(DISTINCT wrp.id) AS number_of_workbench_reaction_patterns
        FROM archive_reaction_pattern AS arp
            LEFT JOIN workbench_reaction_pattern_archive AS wrpa ON arp.id = wrpa.archive_reaction_pattern_id
            LEFT JOIN workbench_reaction_pattern AS wrp ON wrpa.workbench_reaction_pattern_id = wrp.id;
    """
).fetchone()

print((
    "Number of Archive Chemical Reaction Patterns: {number_of_archive_reaction_patterns:,d}\n"
    "Number of Utilized Archive Chemical Reaction Patterns: {number_of_utilized_archive_reaction_patterns:,d}\n"
    "Percentage of Utilized Archive Chemical Reaction Patterns: {percentage_of_utilized_archive_reaction_patterns:f}\n"
    "Number of Workbench Chemical Reaction Patterns: {number_of_workbench_reaction_patterns:,d}"
).format(
    number_of_archive_reaction_patterns=select_statement_result[0],
    number_of_utilized_archive_reaction_patterns=select_statement_result[1],
    percentage_of_utilized_archive_reaction_patterns=select_statement_result[2],
    number_of_workbench_reaction_patterns=select_statement_result[3]
))

Number of Archive Chemical Reaction Patterns: 2,882
Number of Utilized Archive Chemical Reaction Patterns: 2,882
Percentage of Utilized Archive Chemical Reaction Patterns: 100.000000
Number of Workbench Chemical Reaction Patterns: 2,882


In [10]:
for select_statement_parameters_index, select_statement_parameters in enumerate([
    ("'v_retro_transform_db_by_20180421_avramova_s_et_al'", "('')", ),
    ("'v_dingos_by_20190701_button_a_et_al'", "('v_retro_transform_db_by_20180421_avramova_s_et_al')", ),
    ("'v_auto_template_by_20240627_chen_l_and_li_y'", "('v_retro_transform_db_by_20180421_avramova_s_et_al', 'v_dingos_by_20190701_button_a_et_al')", ),
]):
    select_statement_result = case_study_cacs_db.execute_select_statement(
        select_statement_text="""
            SELECT
                COUNT(DISTINCT arp_1.id) AS number_of_archive_reaction_patterns,
                COUNT(DISTINCT wrpa_1.archive_reaction_pattern_id) AS number_of_utilized_archive_reaction_patterns
            FROM archive_reaction_pattern AS arp_1
                JOIN archive_reaction_pattern_source AS arps_1 ON arp_1.id = arps_1.archive_reaction_pattern_id
                JOIN archive_source AS arso_1 ON arps_1.archive_source_id = arso_1.id
                LEFT JOIN workbench_reaction_pattern_archive AS wrpa_1 ON arp_1.id = wrpa_1.archive_reaction_pattern_id
            WHERE arso_1.version = {included_data_source_version:s} AND arp_1.id NOT IN (
                SELECT
                    arps_2.archive_reaction_pattern_id
                FROM archive_reaction_pattern AS arp_2
                    JOIN archive_reaction_pattern_source AS arps_2 ON arp_2.id = arps_2.archive_reaction_pattern_id
                    JOIN archive_source AS arso_2 ON arps_2.archive_source_id = arso_2.id
                WHERE arso_2.version IN {excluded_data_source_versions:s}
            );
        """.format(
            included_data_source_version=select_statement_parameters[0],
            excluded_data_source_versions=select_statement_parameters[1]
        )
    ).fetchone()

    print((
        "Data Source: miscellaneous_{data_source_version:s}\n"
        "Number of Archive Chemical Reaction Patterns: {number_of_archive_reaction_patterns:,d}\n"
        "Number of Utilized Archive Chemical Reaction Patterns: {number_of_utilized_archive_reaction_patterns:,d}\n"
        "Percentage of Utilized Archive Chemical Reaction Patterns: {percentage_of_utilized_archive_reaction_patterns:f}"
    ).format(
        data_source_version=select_statement_parameters[0][1:-1],
        number_of_archive_reaction_patterns=select_statement_result[0],
        number_of_utilized_archive_reaction_patterns=select_statement_result[1],
        percentage_of_utilized_archive_reaction_patterns=100 * select_statement_result[1] / select_statement_result[0]
    ))

    if select_statement_parameters_index < 2:
        print("")

Data Source: miscellaneous_v_retro_transform_db_by_20180421_avramova_s_et_al
Number of Archive Chemical Reaction Patterns: 105
Number of Utilized Archive Chemical Reaction Patterns: 105
Percentage of Utilized Archive Chemical Reaction Patterns: 100.000000

Data Source: miscellaneous_v_dingos_by_20190701_button_a_et_al
Number of Archive Chemical Reaction Patterns: 64
Number of Utilized Archive Chemical Reaction Patterns: 64
Percentage of Utilized Archive Chemical Reaction Patterns: 100.000000

Data Source: miscellaneous_v_auto_template_by_20240627_chen_l_and_li_y
Number of Archive Chemical Reaction Patterns: 2,713
Number of Utilized Archive Chemical Reaction Patterns: 2,713
Percentage of Utilized Archive Chemical Reaction Patterns: 100.000000


In [11]:
for select_statement_parameters_index, select_statement_parameters in enumerate([
    ("'v_retro_transform_db_by_20180421_avramova_s_et_al'", "('')", ),
    ("'v_dingos_by_20190701_button_a_et_al'", "('v_retro_transform_db_by_20180421_avramova_s_et_al')", ),
    ("'v_auto_template_by_20240627_chen_l_and_li_y'", "('v_retro_transform_db_by_20180421_avramova_s_et_al', 'v_dingos_by_20190701_button_a_et_al')", ),
]):
    select_statement_result = case_study_cacs_db.execute_select_statement(
        select_statement_text="""
            SELECT
                COUNT(DISTINCT wrp_1.id) AS number_of_workbench_reaction_patterns
            FROM workbench_reaction_pattern AS wrp_1
                JOIN workbench_reaction_pattern_archive AS wrpa_1 ON wrp_1.id = wrpa_1.workbench_reaction_pattern_id
                JOIN archive_reaction_pattern AS arp_1 ON wrpa_1.archive_reaction_pattern_id = arp_1.id
                JOIN archive_reaction_pattern_source AS arps_1 ON arp_1.id = arps_1.archive_reaction_pattern_id
                JOIN archive_source AS arso_1 ON arps_1.archive_source_id = arso_1.id
            WHERE arso_1.version = {included_data_source_version:s} AND wrp_1.id NOT IN (
                SELECT
                    wrp_2.id
                FROM workbench_reaction_pattern AS wrp_2
                    JOIN workbench_reaction_pattern_archive AS wrpa_2 ON wrp_2.id = wrpa_2.workbench_reaction_pattern_id
                    JOIN archive_reaction_pattern AS arp_2 ON wrpa_2.archive_reaction_pattern_id = arp_2.id
                    JOIN archive_reaction_pattern_source AS arps_2 ON arp_2.id = arps_2.archive_reaction_pattern_id
                    JOIN archive_source AS arso_2 ON arps_2.archive_source_id = arso_2.id
                WHERE arso_2.version IN {excluded_data_source_versions:s}
            );
        """.format(
            included_data_source_version=select_statement_parameters[0],
            excluded_data_source_versions=select_statement_parameters[1]
        )
    ).fetchone()

    print((
        "Data Source: miscellaneous_{data_source_version:s}\n"
        "Number of Workbench Chemical Compounds: {number_of_workbench_reaction_patterns:,d}"
    ).format(
        data_source_version=select_statement_parameters[0][1:-1],
        number_of_workbench_reaction_patterns=select_statement_result[0]
    ))

    if select_statement_parameters_index < 2:
        print("")

Data Source: miscellaneous_v_retro_transform_db_by_20180421_avramova_s_et_al
Number of Workbench Chemical Compounds: 105

Data Source: miscellaneous_v_dingos_by_20190701_button_a_et_al
Number of Workbench Chemical Compounds: 64

Data Source: miscellaneous_v_auto_template_by_20240627_chen_l_and_li_y
Number of Workbench Chemical Compounds: 2,713
